In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read CSV Data
data = pd.read_csv("diabetes.csv")

In [ ]:
data.head(10)

In [ ]:
# number of row's and col in data set 
row,col=data.shape
print("Number of Row's in Data :",row)
print("Number of Col's in Data :",col)

In [ ]:
# Check for duplicates and remove them if found
if data.duplicated().sum() > 0:
    print("Found duplicate rows. Removing them...")
    data = data.drop_duplicates()

In [ ]:
# Check for missing values
missing_values = data.isna().sum()
if missing_values.any():
    print(f"Missing values found in {missing_values.index.tolist()}. Consider imputation techniques.")

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
data["Outcome"].value_counts()

In [ ]:
# Plotting For Diabetes Status
labels = ['Non-Diabetic', 'Diabetic']
plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
sns.countplot(data=data, x='Outcome', hue='Outcome', alpha=1)
plt.legend(labels)
plt.title('Histogram For Diabetes Status', size=15)

plt.subplot(1, 2, 2)  
y = data['Outcome'].value_counts()
explode = [0, 0.02]  
plt.pie(y, labels=labels, explode=explode, autopct='%.2f%%')
plt.axis('equal')
plt.legend(labels)
circle = plt.Circle(xy=(0, 0), radius=0.75, facecolor='white', edgecolor='black')
plt.gca().add_artist(circle)
plt.title('Pie Chart for Diabetes Status', size=15)
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
sns.histplot(data['Age'],kde=False,color='skyblue')
plt.title('Histogram For Age', size=15)
plt.show()

In [ ]:
# Function for Stacked Histograms with Outcome Coloring
def plot_stacked_histograms(data, features, figsize=(15, 12)):
 
  num_features = len(features)
  num_rows = (num_features + 3) // 3  

  plt.figure(figsize=figsize)
  for i, feature in enumerate(features):
    row_num = (i // 3) + 1  
    col_num = (i % 3) + 1  

    plt.subplot(num_rows, 3, (row_num - 1) * 3 + col_num)
    sns.histplot(data=data, x=feature, hue="Outcome", multiple="stack", kde=True)
    plt.title(f"{feature} by Outcome")  

  plt.tight_layout()
  plt.show()

In [ ]:
# Calling the function with our data and list of features
features = ['Age', 'DiabetesPedigreeFunction', 'BMI', 'Insulin', 
            'SkinThickness', 'BloodPressure', 'Glucose']
plot_stacked_histograms(data, features)

In [ ]:
# Function for scatter plot to show the distribution of the age VS other features
def plot_scatter_age_distribution(data, features, figsize=(10, 30)):
   
    num_features = len(features)
    num_rows = num_features 

    plt.figure(figsize=figsize)
    for i, feature in enumerate(features):
        plt.subplot(num_rows, 1, i + 1)  
        sns.scatterplot(data=data, x=feature, y="Age")  
        plt.title(f"Age Distribution VS {feature}")

    plt.tight_layout()
    plt.show()

In [ ]:
# Calling the function with our data and list of features
features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
            'Insulin', 'BMI', 'DiabetesPedigreeFunction']  
plot_scatter_age_distribution(data, features)

In [ ]:
# Correlation Matrix
correlation_matrix = data.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Splitting Features and Target Variable
y = data['Outcome'] # Depended
x = data.drop('Outcome', axis=1) # independents

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
print('Length of Training Set :',len(x_train))
print('Length of Testing Set :',len(x_test))

In [ ]:
k_values = [i for i in range (1,40)]
scores = []

for k in k_values:
    knn_model = KNeighborsClassifier(n_neighbors=k)
    score = cross_val_score(knn_model, x, y, cv=10)
    scores.append(np.mean(score))

In [ ]:
# Visualize K vs Accuracy
sns.lineplot(x = k_values, y = scores, marker = 'o')
plt.xlabel("K Values")
plt.ylabel("Accuracy Score")
plt.show()

In [ ]:
best_index = np.argmax(scores)
best_k = k_values[best_index]

knn_model = KNeighborsClassifier(n_neighbors=best_k)
knn_model.fit(x_train, y_train)

In [ ]:
y_pred = knn_model.predict(x_test) 
knn_accuracy = accuracy_score(y_test, y_pred)
knn_accuracy

In [ ]:
# Evaluate The Model
Training_score = accuracy_score(y_train, knn_model.predict(x_train))  
Testing_score = accuracy_score(y_test, y_pred)

# Precision, Recall, F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Classification Report
class_names = ['Non-Diabetic', 'Diabetic']  
report = classification_report(y_test, y_pred, target_names=class_names, output_dict=True)

# Print the relevant metrics
print('Training Score (Accuracy):', Training_score)
print('Testing Score (Accuracy):', Testing_score)
print('Precision:', precision)
print('Recall:', recall)
print('F1-Score:', f1)

# Print detailed Classification Report
print('\nClassification Report:')
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(pd.DataFrame(report).iloc[:-1, :].T, annot=True, cmap='Blues', linewidths=.5)
plt.title('Classification Report for KNN')
plt.show()

In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["Non-Diabetic", "Diabetic"])
cm_display.plot()
plt.show()

In [ ]:
#loop data use model to predict the outcome
def predict_diabetes(data, model):
    data = scaler.transform(data)
    prediction = model.predict(data)
    return prediction

# Create a sample data

sample_data = [[6, 148, 72, 35, 0, 33.6, 0.627, 50],
               [1, 85, 66, 29, 0, 26.6, 0.351, 31],
                [8, 183, 64, 0, 0, 23.3, 0.672, 32],
                [1, 89, 66, 23, 94, 28.1, 0.167, 21],
                [0, 137, 40, 35, 168, 43.1, 2.288, 33]]

# Predict the outcome
predictions = predict_diabetes(sample_data, knn_model)
for i, prediction in enumerate(predictions):
    print(f"Prediction for Sample {i+1}: {'Diabetic' if prediction == 1 else 'Non-Diabetic'}")
    
# Save the model
import joblib
joblib.dump(knn_model, 'knn_diabetes_model.pkl')
print("Model Saved Successfully!")

# Load the model
loaded_model = joblib.load('knn_diabetes_model.pkl')

# Predict the outcome
predictions = predict_diabetes(sample_data, loaded_model)
for i, prediction in enumerate(predictions):
    print(f"Prediction for Sample {i+1}: {'Diabetic' if prediction == 1 else 'Non-Diabetic'}")
    
# Save the scaler
joblib.dump(scaler, 'scaler.pkl')
print("Scaler Saved Successfully!")


In [50]:
import joblib

sample_data = [[6, 148, 72, 35, 0, 33.6, 0.627, 50],
               [1, 85, 66, 29, 0, 26.6, 0.351, 31],
                [8, 183, 64, 0, 0, 23.3, 0.672, 32],
                [1, 89, 66, 23, 94, 28.1, 0.167, 21],
                [0, 137, 40, 35, 168, 43.1, 2.288, 33]]

loaded_model = joblib.load('knn_diabetes_model.pkl')

predictions = predict_diabetes(sample_data, loaded_model)
for i, prediction in enumerate(predictions):
    print(f"Prediction for Sample {i+1}: {'Diabetic' if prediction == 1 else 'Non-Diabetic'}")

Prediction for Sample 1: Diabetic
Prediction for Sample 2: Non-Diabetic
Prediction for Sample 3: Diabetic
Prediction for Sample 4: Non-Diabetic
Prediction for Sample 5: Diabetic
